# Self Balancing Trees

## Standard Binary Seach Tree

In [41]:
class BSTNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None
    def __str__(self):
        return str(self.val)
        

class BSTree:
    def __init__(self, root):
        self.root = root
    def insert(self, val):
        self.__recursiveInsert(self.root, BSTNode(val)) 
    def __recursiveInsert(self, current, new):
        if(current.val > new.val):
            if(current.left == None):
                current.left = new
                return
            self.__recursiveInsert(current.left, new)
        else:
            if(current.right == None):
                current.right = new
                return
            self.__recursiveInsert(current.right, new)
            
    def find(self, val):
        return self.__recursiveFind(self.root, val)
        
    def __recursiveFind(self, current, val):
        if(current == None):
            return None
        if(current.val == val):
            return current
        if(current.val > val):
            return self.__recursiveFind(current.left, val)
        else:
            return self.__recursiveFind(current.right, val)
        
    def getLengthAtNode(self, start):
        return self.__recursiveGetLengthAtNode(start, 1)
    
    def __recursiveGetLengthAtNode(self, current, level):
        if(current == None):
            return level - 1
        l = self.__recursiveGetLengthAtNode(current.left, level + 1)
        r = self.__recursiveGetLengthAtNode(current.right, level + 1)
        return max(l,r)
    
    def findMin(self):
        return self.__recursiveFindMin(self.root)
    
    def __recursiveFindMin(self, current):
        if(current.left == None):
            return current
        return self.__recursiveFindMin(current.left)
    
    def findMax(self):
        return self.__recursiveFindMax(self.root)
    
    def __recursiveFindMax(self, current):
        if(current.right == None):
            return current
        return self.__recursiveFindMax(current.right)
    
    def findParent(self, node):
        if(node == self.root):
            return (None, False)
        return self.__recursiveFindParent(self.root, node)
    
    def __recursiveFindParent(self, current, node):
        
        l = None
        if(current.left != None):
            if(current.left == node):
                return (current, False)
            l,isRight = self.__recursiveFindParent(current.left, node)
        
        #only return if something is found, otherwise search right
        if(l != None):
            return l,isRight
        
        r = None
        if(current.right != None):
            if(current.right == node):
                return (current, True)
            r,isRight = self.__recursiveFindParent(current.right, node)
            
        if(r != None):
            return r,isRight
         
        return (None, False)
    
    def delete(self, val):
        #finding node that will be deleted
        relevantNode = self.find(val)
        if(relevantNode == None):
            return None
        #parent of node that will be deleted
        parent, isRight = self.findParent(relevantNode)
        
        #if it is a leaf, then simply remove reference
        if(relevantNode.left == None and relevantNode.right == None):
            self.addNewChild(parent, None, isRight)
            return None
        
        #if it is has one None value, set parent to non None reference
        if(relevantNode.left == None):
            self.addNewChild(parent, relevantNode.right, isRight)
            return None
        
        if(relevantNode.right == None):
            self.addNewChild(parent, relevantNode.left, isRight)
            return None
        
        #Since both sides are "heavy", a replacement node must be found
        #replacement node can be either of the 'center' nodes(largest in left subtree or smallest in right subtree) 
        
        replacementNode = None
        #left subtree is bigger, so max of left side will be replacing
        if(self.getLengthAtNode(relevantNode.left) > self.getLengthAtNode(relevantNode.right)):
            
            replacementNode = self.__recursiveFindMax(relevantNode.left) #finds right most node
            maxParent, _ = self.findParent(replacementNode) #has to be on the right side of parent since it is right most node
            
            #if the maxParent == node to be deleted, this replacement still has to occur
            replacementNode.right = relevantNode.right
            
            #this will mostly be true, maxParent == node is an edge case
            if(maxParent != relevantNode):
                #fixing the max parent's right reference to point to maxNodes left children so the tree continuity continues
                maxParent.right = replacementNode.left
                
                #finishing off replacementNodes children references
                replacementNode.left = relevantNode.left
        else:
            replacementNode = self.__recursiveFindMin(relevantNode.right)
            minParent, _ = self.findParent(replacementNode)
            
            replacementNode.left = relevantNode.left
            if(minParent != relevantNode):
                minParent.left = replacementNode.right
                replacementNode.right = relevantNode.right
            
        #updating parent of relevantNode, now it is truly deleted
        self.addNewChild(parent, replacementNode, isRight)
        
        #edge case: root is being deleted
        if(self.root == relevantNode):
            self.root = replacementNode
            
        return relevantNode

            
    #toRight specifies which child to update
    def addNewChild(self,parent, child, toRight):
        if(parent == None):
            return
        if(not toRight):
            parent.left = child
        else:
            parent.right = child
        
    #2d level by level array for printing
    def __generateArray(self):
        queue = [self.root]
        stringArrays = []
        tLevels = self.getLengthAtNode(self.root)
        cLevel = 0
        maxDigits = 0 #max digits of a number, making each number take up the exact same amount of space
        while(cLevel < tLevels):
            cLen = len(queue)
            currStringArray = []
            i = 0
            while(i < cLen):
                curr = queue.pop(0)
                #Nones are being inserted so that all leaf nodes are in the correct positions
                if(curr == None):
                    currStringArray.append("N")
                else:
                    strVal = str(curr.val)
                    currStringArray.append(strVal)
                    cDigitLen = len(strVal)
                    if(cDigitLen > maxDigits):
                        maxDigits = cDigitLen
                
                queue.append(curr.left if curr else None)
                queue.append(curr.right if curr else None)
                i += 1
            stringArrays.append(currStringArray)
            cLevel += 1

        return (stringArrays, maxDigits) 

    def levelPrintTree(self, spacing = 4):
        self.__levelPrintTree(spacing)
    def __levelPrintTree(self, spacing):
        strArrays, maxDigits = self.__generateArray()
        #everything will be spaced relative to the very last line
        lastLineLength = (maxDigits + spacing) * len(strArrays[-1])
        for cLevel in strArrays:
            cPadding = lastLineLength // len(cLevel)
            for item in cLevel:
                print(item.center(maxDigits).center(cPadding),end="")
            print()
        
    def DFSprintTree(self):
        self.__DFSrecursePrint(self.root,0)
        print()
    def __DFSrecursePrint(self, current, lvl):
            if(current == None):
                return
            print(str(lvl) + "-"+str(current.val),end=" ")
            if(current.left != None):
                print("L",end="")
                self.__DFSrecursePrint(current.left, lvl+1)
            if(current.right != None):
                print("R",end="")
                self.__DFSrecursePrint(current.right, lvl+1)


In [51]:
rootNode = BSTNode(4)
tree = BSTree(rootNode)
#[1,4,2,5,6,7]
arr = [2,6,1,3,5,7]
for elem in arr:
    tree.insert(elem)
tree.levelPrintTree()
tree.DFSprintTree()
print("find 5:",tree.find(5))
print("find -1:",tree.find(-1))
print("root length:",tree.getLengthAtNode(rootNode))
print("left subtree length:",tree.getLengthAtNode(rootNode.left))
print("right subtree length:",tree.getLengthAtNode(rootNode.right))
print("min val:",tree.findMin())
print("max val:",tree.findMax())
parent, isRight = tree.findParent(rootNode.right)
print("parent of rootNode's right child:",parent)
print("should be true:",isRight)
tree.delete(4)
print("after deleting 4:")
tree.levelPrintTree()
tree.DFSprintTree()

         4          
    2         6     
  1    3    5    7  
0-4 L1-2 L2-1 R2-3 R1-6 L2-5 R2-7 
find 5: 5
find -1: None
root length: 3
left subtree length: 2
right subtree length: 2
min val: 1
max val: 7
parent of rootNode's right child: 4
should be true: True
after deleting 4:
         5          
    2         6     
  1    3    N    7  
0-5 L1-2 L2-1 R2-3 R1-6 R2-7 


## Purpose of Self Balancing Binary Search Trees
In order to ensure a binary search tree is effective, it needs to be balanced. Self balancing trees enforce certain rules when new elements are added that balance the tree.

## AVL Trees
* A form of Binary Search Trees. No duplicate elements allowed.
* The difference between the height of the left subtree and the height of the right subtree(balance factor) is either -1, 0, or 1